## Lesson 2: Connecting with a CRM

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: Thank you for reaching out, Mike. To process your refund request for
       the broken mug, I will need to transfer you to one of our
       customer service representatives. They will be able to access
       your account details and initiate the refund process. Please
       hold while I transfer your call. We appreciate your patience
       and understanding.

Session ID: 215c8e01-ca29-48bd-986e-771fa3164ff8


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined in the video is at  <code>./ro_shared_data/functions/lambda_stage_1.py</code> </p>

In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': 'd5b6ea23-5575-4a5c-bd1d-a5197e1549f6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 25 Aug 2025 15:07:55 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd5b6ea23-5575-4a5c-bd1d-a5197e1549f6',
   'x-amz-apigw-id': 'P3hbyG9kPHcEJ6A=',
   'x-amzn-trace-id': 'Root=1-68ac7c4b-50039bd32d99ee3e63fe4a28'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:905418467178:function:dlai-support-agent-EXCAG'},
  'actionGroupId': 'SQPN7UOF9Z',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'VQ8V4MPFAR',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 8, 25, 15, 7, 55, 86318, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private 

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']

In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: I apologize, but I don't have enough details to process your refund
       request for the broken mug. Could you please provide the
       following information:  - When and where did you purchase the
       mug? - Do you have the order number or a receipt? - Can you
       describe the issue with the mug in more detail?  With this
       additional information, I can better assist you with getting a
       refund or replacement. Please let me know the details, and I'll
       be happy to help further.

Session ID: cb2c0c6f-7d4c-4f58-be4b-8c22cacb7f7e


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  Okay, let's review the information provided again:  - The customer's
  name is Mike and their email is mike@mike.com - The customer is
  stating their mug is broken and they want a refund  However, I still
  do not have enough details to properly process this refund request.
  I will need to ask the customer for more information before I can
  escalate this to the support team.

Observation:
  Type: FINISH

Final response:
  I apologize, but I still don't have enough details to process your
  refund request for the broken mug. Could you please provide the
  following additional information:  - When and where did you purchase
  the mug? - Do you have the order number or a receipt? - Can you
  describe the issue with the mug in more detail?  With this extra
  information, I can better assist you with getting a refund or
  replacement. Please let me know the details, and I'll b